# Strategy 2 (REFINED): Lightweight Dense MLP

**Refinements Based on Execution Analysis:**

## Issues Found in Original:
- **EXTREME overfitting**: Best at epoch 4, val log2-MSE: 1.270
- **Worse than Strategy 1** which was already bad
- **3x augmentation made it worse**, not better
- **Still too complex**: 4M parameters for simple MLP

## NEW Architecture (Ultra-Simplified):
- **Pure feedforward** (no attention, no residuals)
- **Smaller width**: 512 max (not 1024)
- **Fewer layers**: 512→384→256→128
- **Higher dropout**: 0.5→0.4→0.3
- **Strong L2 regularization**
- **NO augmentation** - original data only

## Strategy:
- Split train/val FIRST
- NO data augmentation
- Simpler than Strategy 1 (~1.5M parameters)
- Goal: Better generalization through simplicity

In [ ]:
import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import (
    Input, Dense, Embedding, Flatten, Concatenate,
    Dropout, BatchNormalization, Lambda
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import (
    EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
)
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("="*70)
print("STRATEGY 2 REFINED: LIGHTWEIGHT DENSE MLP")
print("="*70)
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## Step 1: Load Data

In [ ]:
print("\nSTEP 1: Loading Data")
print("-"*70)

with open('data/combined_final_n_k_m_P.pkl', 'rb') as f:
    inputs_raw = pickle.load(f)

with open('data/combined_final_mHeights.pkl', 'rb') as f:
    outputs_raw = pickle.load(f)

print(f"Raw samples: {len(inputs_raw)}")
print(f"Sample: n={inputs_raw[0][0]}, k={inputs_raw[0][1]}, m={inputs_raw[0][2]}, P={inputs_raw[0][3].shape}")
print(f"Target range: [{np.min(outputs_raw):.2f}, {np.max(outputs_raw):.2f}]")

## Step 2: Split Data (NO AUGMENTATION)

In [ ]:
print("\nSTEP 2: Split Data (NO AUGMENTATION - prevent overfitting)")
print("-"*70)

# Create stratification labels
stratify_labels = [sample[1] * 10 + sample[2] for sample in inputs_raw]

# Split data
inputs_train, inputs_val, outputs_train, outputs_val = train_test_split(
    inputs_raw, outputs_raw,
    test_size=0.15,
    random_state=42,
    stratify=stratify_labels
)

print(f"Training samples: {len(inputs_train)}")
print(f"Validation samples: {len(inputs_val)}")
print("\n✅ NO DATA AUGMENTATION - Simpler is better for this problem")

## Step 3: Prepare Data for Training

In [ ]:
print("\nSTEP 3: Prepare Data for Training")
print("-"*70)

def prepare_data(inputs, outputs):
    n_vals = []
    k_vals = []
    m_vals = []
    P_flat = []
    
    for sample in inputs:
        n_vals.append(sample[0])
        k_vals.append(sample[1])
        m_vals.append(sample[2])
        P_flat.append(sample[3].flatten())
    
    n_vals = np.array(n_vals, dtype=np.float32).reshape(-1, 1)
    k_vals = np.array(k_vals, dtype=np.int32).reshape(-1, 1)
    m_vals = np.array(m_vals, dtype=np.int32).reshape(-1, 1)
    outputs_arr = np.array(outputs, dtype=np.float32)
    
    # Pad P matrices
    max_p_size = max(len(p) for p in P_flat)
    P_padded = []
    for p in P_flat:
        if len(p) < max_p_size:
            padded = np.zeros(max_p_size, dtype=np.float32)
            padded[:len(p)] = p
            P_padded.append(padded)
        else:
            P_padded.append(p)
    
    P_arr = np.array(P_padded, dtype=np.float32)
    outputs_arr = np.maximum(outputs_arr, 1.0)
    
    return n_vals, k_vals, m_vals, P_arr, outputs_arr

n_train, k_train, m_train, P_train, y_train = prepare_data(inputs_train, outputs_train)
n_val, k_val, m_val, P_val, y_val = prepare_data(inputs_val, outputs_val)

# Normalize P matrices (fit on training, transform both)
scaler = StandardScaler()
P_train = scaler.fit_transform(P_train)
P_val = scaler.transform(P_val)

print(f"Training: n={n_train.shape}, k={k_train.shape}, m={m_train.shape}, P={P_train.shape}, y={y_train.shape}")
print(f"Validation: n={n_val.shape}, k={k_val.shape}, m={m_val.shape}, P={P_val.shape}, y={y_val.shape}")
print(f"P train: mean={P_train.mean():.4f}, std={P_train.std():.4f}")
print(f"P val: mean={P_val.mean():.4f}, std={P_val.std():.4f}")

## Step 4: Skipped (No Augmentation)

In [ ]:
print("\nSTEP 4: Skipped (removed augmentation step)")

## Step 5: Build Ultra-Simple Dense MLP

In [ ]:
print("\nSTEP 5: Building Lightweight Dense MLP")
print("-"*70)

def build_model(p_shape, k_vocab_size=7, m_vocab_size=6):
    """
    ULTRA-SIMPLIFIED Dense MLP:
    - NO attention
    - NO residual connections
    - Smaller width: 512 max (not 1024)
    - Fewer layers: 512→384→256→128
    - Very high dropout + L2 regularization
    """
    l2_reg = regularizers.l2(2e-4)  # Stronger L2
    
    n_input = Input(shape=(1,), name='n')
    k_input = Input(shape=(1,), name='k', dtype=tf.int32)
    m_input = Input(shape=(1,), name='m', dtype=tf.int32)
    P_input = Input(shape=(p_shape,), name='P_flat')

    # Embeddings
    k_embed = Flatten()(Embedding(k_vocab_size, 24, embeddings_regularizer=l2_reg)(k_input))
    m_embed = Flatten()(Embedding(m_vocab_size, 24, embeddings_regularizer=l2_reg)(m_input))

    # Simple P processing (no attention)
    x = Dense(192, activation='gelu', kernel_regularizer=l2_reg)(P_input)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)  # Very high dropout
    
    x = Dense(256, activation='gelu', kernel_regularizer=l2_reg)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    # Combine all features
    combined = Concatenate()([n_input, k_embed, m_embed, x])

    # Simple dense progression: 512→384→256→128
    x = Dense(512, activation='gelu', kernel_regularizer=l2_reg)(combined)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(384, activation='gelu', kernel_regularizer=l2_reg)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    
    x = Dense(256, activation='gelu', kernel_regularizer=l2_reg)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(128, activation='gelu', kernel_regularizer=l2_reg)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    # Log-space prediction
    log2_pred = Dense(1, activation='linear', kernel_regularizer=l2_reg)(x)
    log2_positive = Lambda(lambda z: tf.nn.softplus(z))(log2_pred)
    m_height = Lambda(lambda z: tf.pow(2.0, z))(log2_positive)

    return Model(inputs=[n_input, k_input, m_input, P_input], outputs=m_height,
                 name='strategy2_lightweight')

model = build_model(P_train.shape[1], k_vocab_size=k_train.max()+1, m_vocab_size=m_train.max()+1)
print(f"Parameters: {model.count_params():,} (vs 4M in original)")
model.summary()

## Step 6: Train with Strong Regularization

In [ ]:
print("\nSTEP 6: Compile and Train")
print("-"*70)

def log2_mse_loss(y_true, y_pred):
    epsilon = 1e-7
    y_true = tf.maximum(y_true, epsilon)
    y_pred = tf.maximum(y_pred, epsilon)
    log2_true = tf.math.log(y_true) / tf.math.log(2.0)
    log2_pred = tf.math.log(y_pred) / tf.math.log(2.0)
    return tf.reduce_mean(tf.square(log2_true - log2_pred))

# Lower learning rate, stronger weight decay
optimizer = AdamW(learning_rate=3e-4, weight_decay=3e-4, clipnorm=1.0)
model.compile(optimizer=optimizer, loss=log2_mse_loss, metrics=[log2_mse_loss])

# Aggressive early stopping
callbacks = [
    EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=12, min_lr=1e-6, verbose=1),
    ModelCheckpoint('strategy2_refined_best.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

print("Training with NO augmentation, maximum regularization, minimal architecture...")
history = model.fit(
    [n_train, k_train, m_train, P_train], y_train,
    validation_data=([n_val, k_val, m_val, P_val], y_val),
    epochs=150, batch_size=256, callbacks=callbacks, verbose=1
)

print("\nTraining completed!")

## Step 7: Evaluate

In [ ]:
print("\nSTEP 7: Evaluation")
print("-"*70)

model.load_weights('strategy2_refined_best.h5')
y_pred_val = model.predict([n_val, k_val, m_val, P_val], verbose=0).flatten()

def compute_log2_mse(y_true, y_pred):
    epsilon = 1e-7
    y_true = np.maximum(y_true, epsilon)
    y_pred = np.maximum(y_pred, epsilon)
    return np.mean((np.log2(y_true) - np.log2(y_pred)) ** 2)

val_log2_mse = compute_log2_mse(y_val, y_pred_val)

print(f"\nValidation log2-MSE: {val_log2_mse:.6f}")
print(f"Prediction range: [{y_pred_val.min():.2f}, {y_pred_val.max():.2f}]")
print(f"\nCompare to original: 1.270 (peaked at epoch 4!)")

group_metrics = defaultdict(lambda: {'true': [], 'pred': []})
for i in range(len(y_val)):
    k, m = k_val[i, 0], m_val[i, 0]
    group_metrics[(k, m)]['true'].append(y_val[i])
    group_metrics[(k, m)]['pred'].append(y_pred_val[i])

print("\nPer-Group Performance:")
print(f"{'Group':<12} {'n_val':<8} {'log2-MSE':<12} {'vs Original':<15}")
print("-"*55)

# Original per-group results for comparison
orig_results = {
    (4,2): 0.356, (4,3): 0.379, (4,4): 0.866, (4,5): 2.627,
    (5,2): 0.340, (5,3): 0.935, (5,4): 2.633,
    (6,2): 0.715, (6,3): 2.577
}

for (k, m), data in sorted(group_metrics.items()):
    group_mse = compute_log2_mse(np.array(data['true']), np.array(data['pred']))
    orig_mse = orig_results.get((k, m), 0)
    diff = group_mse - orig_mse
    sign = "+" if diff > 0 else ""
    print(f"k={k}, m={m}    {len(data['true']):6d}   {group_mse:.6f}      {sign}{diff:.3f}")

print("\n" + "="*70)
print("STRATEGY 2 REFINED COMPLETE")
print("="*70)